# Multi-view reconstruction and Meshing

## 0. Import

In [ ]:
%matplotlib inline

import cv2

import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display as phm_display
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.multi_view_reconstruction as phm_mvr
import openalea.phenomenal.mesh as phm_mesh
import openalea.phenomenal.display.notebook as phm_display_notebook

## 1. Prerequisites

### 1.1 Load data

In [ ]:
plant_number = 2  # Available : 1, 2, 3, 4 or 5
bin_images = phm_data.bin_images(f"data/plant_{plant_number}")
calibrations = phm_data.calibrations(f"data/plant_{plant_number}")

phm_display.show_images(
    list(bin_images["side"].values()) + list(bin_images["top"].values())
)

## 2. Multi-view reconstruction

### 2.1 Associate images and projection function

In [ ]:
def routine_select_ref_angle(bin_side_images):
    max_len = 0
    max_angle = None

    for angle in bin_side_images:
        x_pos, y_pos, x_len, y_len = cv2.boundingRect(
            cv2.findNonZero(bin_side_images[angle])
        )

        if x_len > max_len:
            max_len = x_len
            max_angle = angle

    return max_angle

In [ ]:
refs_angle_list = [routine_select_ref_angle(bin_images["side"])]
print(refs_angle_list)

image_views = list()
for id_camera in bin_images:
    for angle in bin_images[id_camera]:
        projection = calibrations[id_camera].get_projection(angle)

        image_ref = None
        if id_camera == "side" and angle in refs_angle_list:
            image_ref = bin_images[id_camera][angle]

        inclusive = False
        if id_camera == "top":
            inclusive = True

        image_views.append(
            phm_obj.ImageView(
                bin_images[id_camera][angle],
                projection,
                inclusive=inclusive,
                image_ref=image_ref,
            )
        )

### 2.2 Do multi-view reconstruction

In [ ]:
voxels_size = 16  # mm
error_tolerance = 0
voxel_grid = phm_mvr.reconstruction_3d(
    image_views, voxels_size=voxels_size, error_tolerance=error_tolerance
)

### 2.4 Save / Load voxel grid

In [ ]:
voxel_grid.write(f"plant_{plant_number}_size_{voxels_size}.npz")

In [ ]:
voxel_grid = phm_obj.VoxelGrid.read(
    f"plant_{plant_number}_size_{voxels_size}.npz"
)

### 2.5 Viewing

In [ ]:
phm_display_notebook.show_voxel_grid(voxel_grid, size=1)

## 3.Meshing

In [ ]:
vertices, faces = phm_mesh.meshing(
    voxel_grid.to_image_3d(), reduction=0.90, smoothing_iteration=5, verbose=True
)

print("Number of vertices : {nb_vertices}".format(nb_vertices=len(vertices)))
print("Number of faces : {nb_faces}".format(nb_faces=len(faces)))

### Viewing

In [ ]:
phm_display_notebook.show_mesh(vertices, faces)